## Pre processing

In [32]:
from amplpy import modules
import pandas as pd
import pyomo.environ as pyo
import numpy as np

In [33]:
# Read CSV file, disaggregated demand (3 consumers)
cons_df = pd.read_csv("Summer_Berlin_Disaggregated_3Consumers.csv", sep=';')

In [34]:
# Identify the 3 consumer columns and check if they exist in the excel file.
consumer_cols = ["consumer_1", "consumer_2", "consumer_3"]
for c in consumer_cols:
    assert c in cons_df.columns, f"Missing column {c} in disaggregated file."

# Define T as the number of time steps based on the length of the demand dataframe
T = len(cons_df)
t_idx = range(T)

# The D_ct dictionary is currently not used in the Pyomo model for m.D,
# as m.D uses an aggregate demand. It can be ignored or removed for now.
# D_ct = {(c, t): float(cons_df.loc[t, c]) for c in consumer_cols for t in t_idx}

In [35]:
# Read CSV file : PV and market price and check the number of rows
pv_df = pd.read_csv("Berlin_Summer_Week_01.07_07.07_2023.csv")
price_df = pd.read_csv("Hourly_Market_Prize_Berlin_Summer_Week.csv")

assert len(pv_df) == T, "PV file length != demand length"
assert len(price_df) == T, "Price file length != demand length"

In [36]:
# Build (c,t) demand dictionary for Pyomo Param ( instead of a vector we build a matric , note : this is only for demand
# because [we have consumer and time])
D_ct = {(c, t): float(cons_df.loc[t, c]) for c in consumer_cols for t in t_idx}

In [37]:
# PV scaling : W -> kW for reference plant, # kW per 1 kWp installed
pv_ref_kwp = 1000.0
pv_df["pv_kw"] = pv_df["Power"].astype(float) / 1000.0
pv_df["pv_per_kwp"] = pv_df["pv_kw"] / pv_ref_kwp
# convert to vector for PV and market price
PV_unit = pv_df["pv_per_kwp"].to_numpy(dtype=float)
P_buy   = price_df["price_eur_per_kwh"].to_numpy(dtype=float)
# build new vector
sell_factor = 0.50
P_sell = sell_factor * P_buy


In [38]:
# fix the size of PV and battery
PV_cap_opt = 2429.59124087591
E_cap_opt  = 177.769887208029

# Battery technical parameters
eta = 0.90
soc_min_frac  = 0.10
soc_max_frac  = 0.90
soc_init_frac = 0.50
soc_final_frac= 0.50
c_rate = 1.0
P_cap_opt = c_rate * E_cap_opt  # 1C

## Building the model

In [39]:
m = pyo.ConcreteModel()

In [40]:
# Sets
m.T = pyo.RangeSet(0, T-1)
m.Tsoc = pyo.RangeSet(0, T)
# defining a set for consumers
m.C = pyo.Set(initialize=consumer_cols)

In [41]:
#parameters :
m.D = pyo.Param(m.C, m.T, initialize=D_ct)
m.PV_unit = pyo.Param(m.T, initialize={t: float(PV_unit[t]) for t in t_idx})
m.p_buy   = pyo.Param(m.T, initialize={t: float(P_buy[t]) for t in t_idx})
m.p_sell  = pyo.Param(m.T, initialize={t: float(P_sell[t]) for t in t_idx})

# Fixed sizes as Params
m.PV_cap = pyo.Param(initialize=float(PV_cap_opt))
m.E_cap  = pyo.Param(initialize=float(E_cap_opt))
m.P_cap  = pyo.Param(initialize=float(P_cap_opt))


In [42]:
#Decision Variables
# PV allocations
#PV to each consumer
m.pv2load = pyo.Var(m.C, m.T, domain=pyo.NonNegativeReals)
#pv to battery / and PV exported to the grid
m.pv2batt = pyo.Var(m.T, domain=pyo.NonNegativeReals)
m.pv_exp  = pyo.Var(m.T, domain=pyo.NonNegativeReals)
# grid to load / grid to battery
m.g2load = pyo.Var(m.C, m.T, domain=pyo.NonNegativeReals)
m.g2batt = pyo.Var(m.T, domain=pyo.NonNegativeReals)
# battery and state of charge
m.ch  = pyo.Var(m.T, domain=pyo.NonNegativeReals)
m.dis = pyo.Var(m.T, domain=pyo.NonNegativeReals)
m.soc = pyo.Var(m.Tsoc, domain=pyo.NonNegativeReals)
# Battery discharge allocation to consumers (important for per-consumer results)
m.batt2load = pyo.Var(m.C, m.T, domain=pyo.NonNegativeReals)
# Binary (no simultaneous charge/discharge)
m.u = pyo.Var(m.T, domain=pyo.Binary)

In [43]:
#Constraint
def pv_split_rule(m, t):
    return sum(m.pv2load[c,t] for c in m.C) + m.pv2batt[t] + m.pv_exp[t] == m.PV_unit[t] * m.PV_cap
m.pv_split = pyo.Constraint(m.T, rule=pv_split_rule)

def demand_balance_rule(m, c, t):
    return m.pv2load[c,t] + m.batt2load[c,t] + m.g2load[c,t] == m.D[c,t]
m.demand_balance = pyo.Constraint(m.C, m.T, rule=demand_balance_rule)

# Allocate total discharge to consumers
def dis_alloc_rule(m, t):
    return sum(m.batt2load[c,t] for c in m.C) == m.dis[t]
m.dis_alloc = pyo.Constraint(m.T, rule=dis_alloc_rule)

def charge_def_rule(m, t):
    return m.ch[t] == m.pv2batt[t] + m.g2batt[t]
m.charge_def = pyo.Constraint(m.T, rule=charge_def_rule)

# SOC dynamics
def soc_update_rule(m, t):
    return m.soc[t+1] == m.soc[t] + eta*m.ch[t] - (1/eta)*m.dis[t]
m.soc_update = pyo.Constraint(m.T, rule=soc_update_rule)

# SOC bounds
# another method similar to the function method ( we use it to make the constraint more readable)
m.soc_min = pyo.Constraint(m.Tsoc, rule=lambda m,k: m.soc[k] >= soc_min_frac*m.E_cap)
m.soc_max = pyo.Constraint(m.Tsoc, rule=lambda m,k: m.soc[k] <= soc_max_frac*m.E_cap)

# Initial/final SOC
m.soc_init  = pyo.Constraint(expr=m.soc[0] == soc_init_frac*m.E_cap)
m.soc_final = pyo.Constraint(expr=m.soc[T] == soc_final_frac*m.E_cap)

# Power limits (fixed P_cap)
m.ch_size  = pyo.Constraint(m.T, rule=lambda m,t: m.ch[t]  <= m.P_cap)
m.dis_size = pyo.Constraint(m.T, rule=lambda m,t: m.dis[t] <= m.P_cap)

# No simultaneous charge/discharge
m.ch_onoff  = pyo.Constraint(m.T, rule=lambda m,t: m.ch[t]  <= m.u[t] * m.P_cap)
m.dis_onoff = pyo.Constraint(m.T, rule=lambda m,t: m.dis[t] <= (1-m.u[t]) * m.P_cap)

# No simultaneous charge/discharge
m.ch_onoff  = pyo.Constraint(m.T, rule=lambda m,t: m.ch[t]  <= m.u[t] * m.P_cap)
m.dis_onoff = pyo.Constraint(m.T, rule=lambda m,t: m.dis[t] <= (1-m.u[t]) * m.P_cap)



This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [44]:
#Objective function (we remove the capex parameters)
# fee imposed for grid network services
grid_fee_eur_per_kwh = 0.0
def op_cost_rule(m):
    grid_cost = sum((m.p_buy[t] + grid_fee_eur_per_kwh) *
                    (sum(m.g2load[c,t] for c in m.C) + m.g2batt[t])
                    for t in m.T)
    export_revenue = sum(m.p_sell[t] * m.pv_exp[t] for t in m.T)
    return grid_cost - export_revenue

m.obj = pyo.Objective(rule=op_cost_rule, sense=pyo.minimize)

In [47]:
import pyomo.environ as pyo

# Install glpk-utils if not already installed
!apt-get -qq update
!apt-get -qq install -y glpk-utils

# Configure the GLPK solver with the explicit path to its executable
solver = pyo.SolverFactory("glpk", executable="/usr/bin/glpsol")

# Solve the model
res = solver.solve(m)

# Print the operational cost
print("Operational cost (€) =", pyo.value(m.obj))

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Operational cost (€) = 3284.601980321103
